## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2023-11-16 00:43:05.989780: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

clean_application_df = application_df.drop(columns = ['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS','ASK_AMT'])

clean_application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,1
...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,0
34297,T5,Independent,C3000,ProductDev,Association,0,1


In [3]:
# Determine the number of unique values in each column.

clean_application_df.nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
INCOME_AMT           9
IS_SUCCESSFUL        2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning

app_type = clean_application_df["APPLICATION_TYPE"].value_counts()
app_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = list(app_type.iloc[8:].index)
cutoff_value = 528

# Replace in dataframe
for app in application_types_to_replace:
    clean_application_df['APPLICATION_TYPE'] = clean_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
clean_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning

classification_types = clean_application_df["CLASSIFICATION"].value_counts()
classification_types

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1

classifications_cleaned = classification_types.loc[classification_types > 1]
classifications_cleaned

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = list(classification_types.iloc[5:].index)
cutoff_value_classifications = 1883

# Replace in dataframe
for cls in classifications_to_replace:
    clean_application_df['CLASSIFICATION'] = clean_application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
clean_application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`

dummy = pd.get_dummies(clean_application_df)
dummy.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [11]:
# Split our preprocessed data into our features and target arrays

y = dummy["IS_SUCCESSFUL"].values
X = dummy.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# FIRST ATTEMPT

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

total_input_features = len(X_train[0])
hidden_nodes_l1 = 8
hidden_nodes_l2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_l1, input_dim = total_input_features,activation = "relu"))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_l2, activation="relu"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 8)                 320       
                                                                 
 dense_7 (Dense)             (None, 5)                 45        
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 371 (1.45 KB)
Trainable params: 371 (1.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Compile the model (FIRST ATTEMPT)

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the model (FIRST ATTEMPT)

model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 9s 7ms/step - loss: 0.6367 - accuracy: 0.6573
Epoch 2/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5764 - accuracy: 0.7206
Epoch 3/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5647 - accuracy: 0.7280
Epoch 4/100
804/804 [==============================] - 5s 7ms/step - loss: 0.5613 - accuracy: 0.7282
Epoch 5/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5593 - accuracy: 0.7290
Epoch 6/100
804/804 [==============================] - 8s 9ms/step - loss: 0.5582 - accuracy: 0.7284
Epoch 7/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5577 - accuracy: 0.7286
Epoch 8/100
804/804 [==============================] - 16s 20ms/step - loss: 0.5569 - accuracy: 0.7291
Epoch 9/100
804/804 [==============================] - 10s 12ms/step - loss: 0.5560 - accuracy: 0.7290
Epoch 10/100
804/804 [==============================] - 13s 16ms/step - loss: 0.5550 - 

804/804 [==============================] - 3s 3ms/step - loss: 0.5467 - accuracy: 0.7327
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7343
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7327
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5463 - accuracy: 0.7331
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7341
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7336
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5462 - accuracy: 0.7343
Epoch 88/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5462 - accuracy: 0.7335
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5461 - accuracy: 0.7337
Epoch 90/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5459 - accuracy: 

In [18]:
# Evaluate the model using the test data (FIRST ATTEMPT)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5506 - accuracy: 0.7310 - 451ms/epoch - 2ms/step
Loss: 0.5506063103675842, Accuracy: 0.7309620976448059


In [19]:
# SECOND ATTEMPT

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

total_input_features = len(X_train[0])
hidden_nodes_l1 = 10
hidden_nodes_l2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_l1, input_dim = total_input_features,activation = "relu"))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_l2, activation="relu"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 10)                400       
                                                                 
 dense_10 (Dense)            (None, 10)                110       
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 521 (2.04 KB)
Trainable params: 521 (2.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model (SECOND ATTEMPT)

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model (SECOND ATTEMPT)

model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6104 - accuracy: 0.6966
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5648 - accuracy: 0.7242
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5584 - accuracy: 0.7266
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5561 - accuracy: 0.7278
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5550 - accuracy: 0.7296
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5541 - accuracy: 0.7292
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5534 - accuracy: 0.7292
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5525 - accuracy: 0.7286
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5518 - accuracy: 0.7307
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5513 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.5432 - accuracy: 0.7352
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5433 - accuracy: 0.7343
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5434 - accuracy: 0.7352
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5433 - accuracy: 0.7351
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5434 - accuracy: 0.7350
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5432 - accuracy: 0.7348
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5431 - accuracy: 0.7349
Epoch 88/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5435 - accuracy: 0.7347
Epoch 89/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5431 - accuracy: 0.7358
Epoch 90/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5430 - accuracy: 

In [22]:
# Evaluate the model using the test data (SECOND ATTEMPT)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5505 - accuracy: 0.7312 - 484ms/epoch - 2ms/step
Loss: 0.550543487071991, Accuracy: 0.731195330619812


In [23]:
# THIRD ATTEMPT

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

total_input_features = len(X_train[0])
hidden_nodes_l1 = 25
hidden_nodes_l2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_l1, input_dim = total_input_features,activation = "tanh"))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_l2, activation="sigmoid"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 25)                1000      
                                                                 
 dense_13 (Dense)            (None, 30)                780       
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 1811 (7.07 KB)
Trainable params: 1811 (7.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
# Compile the model (THIRD ATTEMPT)

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model (THIRD ATTEMPT)

model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5913 - accuracy: 0.7107
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5731 - accuracy: 0.7227
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5665 - accuracy: 0.7271
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5643 - accuracy: 0.7283
Epoch 5/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5885 - accuracy: 0.7170
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5620 - accuracy: 0.7281
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5565 - accuracy: 0.7287
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5552 - accuracy: 0.7301
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5734 - accuracy: 0.7196
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5795 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.5446 - accuracy: 0.7362
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5483 - accuracy: 0.7357
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5523 - accuracy: 0.7360
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5480 - accuracy: 0.7343
Epoch 85/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5449 - accuracy: 0.7360
Epoch 86/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5475 - accuracy: 0.7357
Epoch 87/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5443 - accuracy: 0.7364
Epoch 88/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5458 - accuracy: 0.7349
Epoch 89/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5487 - accuracy: 0.7352
Epoch 90/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5439 - accuracy: 

In [26]:
# Evaluate the model using the test data (THIRD ATTEMPT)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6083 - accuracy: 0.7166 - 550ms/epoch - 2ms/step
Loss: 0.608252763748169, Accuracy: 0.7166180610656738


In [27]:
# Export our model to HDF5 file

nn.save("AlphabetSoupCharity_Optimization.h5")

/Users/kelvinramos/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
